## PUBG Modelling

플레이어들이 전장에서 벌이는 모든 행위를 분류하여 승리를 예측하기 위한 프로젝트

## Column 별 정보
- ASSISTS : 자신에 의해 사살되지는 못했지만 자신의 데미지 지분율이 가장 높은 적의 수
- BOOSTS : 도핑 아이템 사용 횟수(에너지 드링크, 진통제, 아드레날린 주사기)
- DAMAGEDEALT : 적에게 가한 전체 데미지
- DBNOS : 기절시킨 적의 수
- GROUPID : 경기 내 그룹 식별 ID
- HEADSHOTKILLS : 헤드샷으로 죽인 적의 수
- HEALS : 회복 아이템 사용 횟수(붕대, 구급상자, 의료용 키트)
- ID : 해당 데이터 ID
- KILLPLACE : 경기 내 적을 죽인 적 수의 순위
- KILLPOINTS : 유저의 ELO레이팅(죽인 적 수 기반)
- KILLS : 경기 내 적을 죽인 수
- KILLSTREAKS : 짧은 시간 내에 연속으로 적을 죽인 수
- LONGESTKILL : 사살한 적까지의 거리의 최대값
- MATCHDURATION : 경기가 진행된 시간(단위:초)
- MATCHID : 경기 식별 ID
- MATCHTYPE : 경기 모드( ex: 솔로, 듀오, 스쿼드 )
- MAXPLACE : 경기 내 총 인원 수
- NUMGROUP : 경기 내 실제 참여 인원 수
- RANKPOINTS : 유저의 ELO 레이팅
- REVIVES : 유저가 팀원을 부활시킨 횟수
- RIDEDISTANCE : 이동수단을 통해 이동한 거리(단위:m)
- ROADKILLS : 이동수단으로 살해한 적의 수
- SWIMDISTANCE : 수영으로 이동한 거리(단위:m)
- TEAMKILLS : 같은 팀원을 살해한 수
- VEHICLEDESTROYS : 파괴한 이동수단의 수
- WALKDISTANCE : 도보로 이동한 거리(단위:m)
- WEAPONSACQUIRED : 획득한 무기의 수
- WINPOINTS : 유저의 ELO 레이팅(승리 횟수 기반)
- WINPLACEPERC : 현재 경기에서의 백분위 기반 유저의 순위(종속변수)

## 라이브러리 및 데이터 불러오기

In [1]:
# 데이터 분석
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# 학습을 위한 라이브러리 세팅
from sklearn.linear_model import LinearRegression   
from sklearn.linear_model import Lasso              
from sklearn.linear_model import Ridge             
from xgboost.sklearn import XGBRegressor            
from lightgbm.sklearn import LGBMRegressor 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMClassifier        
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from functools import partial

# VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Hyper parameter tuning
import optuna
import sklearn.metrics as metrics

# others 
import scipy as sp
from scipy import stats
import sys
import gc
import os
print(os.listdir("/Users/krc/TIL/PJT_modelling"))
import warnings                      
warnings.filterwarnings("ignore")

/Users/krc/miniforge3/envs/ml-modeling/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


['custfunc.py', 'featuring.py', '.DS_Store', 'modelling.py', '__pycache__', '.ipynb_checkpoints', 'processing.py', 'PUBG_Modelling_Final.ipynb', 'main.py', 'data', 'catboost_info']


/Users/krc/miniforge3/envs/ml-modeling/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv('/Users/krc/TIL/PJT_modelling/data/train_V2.csv')

test = pd.read_csv('/Users/krc/TIL/PJT_modelling/data/test_V2.csv')

submission = pd.read_csv('/Users/krc/TIL/PJT_modelling/data/sample_submission_V2.csv')

## 함수

In [3]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
#     start_mem = df.memory_usage().sum() / 1024**2
#     print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

#     end_mem = df.memory_usage().sum() / 1024**2
#     print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
#     print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [4]:
def sorted_corr(data, column) :
    df_corr = data.corr()
    df_corr = df_corr.apply(lambda x: round(x ,2))
    df_corr = df_corr.unstack()
    df_corr = pd.DataFrame(df_corr[column][df_corr[column]<1].sort_values(ascending=False), columns=['Correlation'])
    df_corr = df_corr.style.background_gradient(cmap='coolwarm_r')
    return df_corr

In [5]:
def toVIF(features): 
    return pd.DataFrame({
        "feature": features.columns,
        "VIF": [variance_inflation_factor(features.values, idx)
                for idx in range(features.shape[1])]
    })

In [6]:
def OLS_summary(features, target):
    sm_feature = sm.add_constant(features)
    model = sm.OLS(target, sm_feature).fit()
    print(model.summary2())

In [37]:
from sklearn.linear_model import LinearRegression   # 1. Linear Regression
from sklearn.linear_model import Lasso              # 2. Lasso
from sklearn.linear_model import Ridge              # 3. Ridge
from xgboost.sklearn import XGBRegressor            # 4. XGBoost
from lightgbm.sklearn import LGBMRegressor          # 5. LightGBM
from catboost import CatBoostRegressor              # 6. Catboost / SVR = X

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error


def Linear_training_mae(target, features):
    train_X, val_X, train_y, val_y = train_test_split(features, target, test_size=0.2, random_state=1511)
    model = LinearRegression().fit(train_X, train_y)
    pred_train = model.predict(train_X)
    pred_val = model.predict(val_X)
    mae_train = mean_absolute_error(train_y, pred_train)
    mae_val = mean_absolute_error(val_y, pred_val)
    return print("Linear Regression\t train=%.4f, val=%.4f" % (mae_train, mae_val))
def Lasso_training_mae(target, features):
    train_X, val_X, train_y, val_y = train_test_split(features, target, test_size=0.2, random_state=1511)
    model2 = Lasso().fit(train_X,train_y)
    pred_train2 = model2.predict(train_X)
    pred_val2 = model2.predict(val_X)
    mae_train2 = mean_absolute_error(train_y, pred_train2)
    mae_val2 = mean_absolute_error(val_y, pred_val2)
    return print("Lasso\t\t train=%.4f, val=%4.f" % (mae_train2, mae_val2))
def Ridge_training_mae(target, features):
    train_X, val_X, train_y, val_y = train_test_split(features, target, test_size=0.2, random_state=1511)
    model3 = Ridge().fit(train_X,train_y)
    pred_train3 = model3.predict(train_X)
    pred_val3 = model3.predict(val_X)
    mae_train3 = mean_absolute_error(train_y, pred_train3)
    mae_val3 = mean_absolute_error(val_y, pred_val3)
    return print("Ridge\t\t train=%.4f, val=%.4f" % (mae_train3, mae_val3))
def XGB_training_mae(target, features):
    train_X, val_X, train_y, val_y = train_test_split(features, target, test_size=0.2, random_state=1511)
    model4 = XGBRegressor().fit(train_X, train_y)
    pred_train4 = model4.predict(train_X)
    pred_val4 = model4.predict(val_X)
    mae_train4 = mean_absolute_error(train_y, pred_train4)
    mae_val4 = mean_absolute_error(val_y, pred_val4)
    return print("XGBoost\t\t train=%.4f, val=%.4f" % (mae_train4, mae_val4))
def LGBM_training_mae(target, features):
    train_X, val_X, train_y, val_y = train_test_split(features, target, test_size=0.2, random_state=1511)
    model5 = LGBMRegressor().fit(train_X, train_y)
    pred_train5 = model5.predict(train_X)
    pred_val5 = model5.predict(val_X)
    mae_train5 = mean_absolute_error(train_y, pred_train5)
    mae_val5 = mean_absolute_error(val_y, pred_val5)
    return print("LightGBM\t\t train=%.4f, val=%.4f" % (mae_train5, mae_val5))
def CB_training_mae(target, features):
    train_X, val_X, train_y, val_y = train_test_split(features, target, test_size=0.2, random_state=1511)
    model6 = CatBoostRegressor(iterations=40, depth=16, learning_rate=1, loss_function='MAE').fit(train_X, train_y)
    pred_train6 = model6.predict(train_X)
    pred_val6 = model6.predict(val_X)
    mae_train6 = mean_absolute_error(train_y, pred_train6)
    mae_val6 = mean_absolute_error(val_y, pred_val6)
    return print("CatBoost Regression \t\t train=%.4f, val=%.4f" % (mae_train6, mae_val6))

    
def AllModels_mae(target, features):
    print("Linaer training..")
    Linear = Linear_training_mae(target, features)
    print("Lasso training..")
    Lasso = Lasso_training_mae(target, features)
    print("Ridge training..")
    Ridge = Ridge_training_mae(target, features)
    print("XGBoost training..")
    XGB = XGB_training_mae(target, features)
    print("LightGBM training..")
    LGBM = LGBM_training_mae(target, features)
    print("CatBoost training..")
    CB = CB_training_mae(target, features)
    # return Linear, Lasso, Ridge, XGB, LGBM, CB

In [8]:
def one_hot_encoding(data):
    train_OHE = pd.get_dummies(data, columns=["matchType"])
    train_OHE = reduce_mem_usage(train_OHE)
    return train_OHE

In [9]:
def ordinal_encoding(data):
    train_OE = data.copy()
    train_OE['matchType'] = train_OE['matchType'].map({
        'solo':1,
        'solo-fpp':2,
        'duo':3,
        'duo-fpp':4,
        'squad':5,
        'squad-fpp':6,
        'normal-duo':7,
        'normal-duo-fpp':8,
        'normal-solo':9,
        'normal-solo-fpp':10,
        'normal-squad':11,
        'normal-squad-fpp':12,
        'crashfpp':13,
        'crashtpp':14,
        'flarefpp':15,
        'flaretpp':16
        })
    train_OE = reduce_mem_usage(train_OE)
    return train_OE

In [10]:
# optuna RandomForest
def optimizer_RF(trial, X, y, K):
    n_estimators = trial.suggest_int("n_estimators", 50, 200)
    max_depth = trial.suggest_int("max_depth", 8, 30)
    max_features = trial.suggest_categorical("max_features", ['auto', 'sqrt', 'log2'])
    evaluation_metric = mean_absolute_error
    
    model = RandomForestRegressor(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  max_features=max_features,
                                  n_jobs=-1,
                                  random_state=0xC0FFEE)
    
    folds = KFold(n_splits=K)
    scores = []
    
    for train_idx, val_idx in folds.split(X, y):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]
        
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        score = evaluation_metric(y_val, preds)
        scores.append(score)
        
    return np.mean(scores)

In [11]:
# Optuna LightGBM
def optimizer_LGBM(trial, X, y, K):
    import os
    param = {
        'objective': 'regression', # 회귀
        'verbose': 0,
        'max_depth': trial.suggest_int('max_depth', 8, 20),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1)
        #"device" : 'gpu'
    }

    model = LGBMRegressor(**param, n_jobs=os.cpu_count())
    evaluation_metric = mean_absolute_error
    
    folds = KFold(n_splits=K)
    scores = []
    
    for train_idx, val_idx in folds.split(X, y):
        
        # X_train,X_val = X[train_idx],X[val_idx]
        # y_train,y_val = y[train_idx],y[val_idx]
        
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]
        
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=25)
        preds = model.predict(X_val)
        score = evaluation_metric(y_val, preds)
        scores.append(score)
    
    return np.mean(scores)

In [12]:
def dropIdx(df, idx) :
    df.drop(index=idx, inplace=True)
    dropIdx.dpIdx_sum +=len(idx)
    return df
dropIdx.dpIdx_sum = 0

In [13]:
def dropOutlier (df):
    df = df.dropna(axis=0)
    def dropIdx(df, idx) :
        df.drop(index=idx, inplace=True)
        dropIdx.dpIdx_sum +=len(idx)
        return df
    dropIdx.dpIdx_sum = 0

    print("Pre-Processing...")
    for i in df.columns.to_list() :
        df.drop(index=df[df[i].isnull()==True].index, inplace=True)
        
    print("Droping Outliers...")

    vip_features = ["assists","boosts","DBNOs","heals","kills","killStreaks","walkDistance", "revives", "roadKills", "vehicleDestroys"]

    group = df.groupby('groupId').count()
    df = dropIdx(df, df[df.groupId.isin(group[group["Id"]>group["Id"].quantile(0.9999)].index)==True].index) #수치고려 가능

    for col in (vip_features + ["damageDealt","longestKill", "rideDistance", "swimDistance","weaponsAcquired", "matchDuration"]):
        df = dropIdx(df, df[df[col]>df[col].quantile(0.999)].index)
    
    for col in vip_features:
        df = dropIdx(df, df[df["walkDistance"]<df[col]].index)

    df = dropIdx(df, df[df.groupby('matchId')['kills'].transform('max')  > df.groupby('matchId')['Id'].transform('count')  ].index)
    df = dropIdx(df, df[(df['rideDistance']==0) & (df['roadKills']>0)  ].index)

    #edge case
    df.loc[(df.maxPlace>1)&(df.numGroups==1), "maxPlace"] = 1

    print(f"{dropIdx.dpIdx_sum} Columns has deleted!") 

    del vip_features, group      
    gc.collect()
    
    return df

"assists",
"boosts","DBNOs",
"heals","kills",
"killStreaks","walkDistance", 
"revives", "roadKills", 
"vehicleDestroys","damageDealt",
"longestKill", "rideDistance", 
"swimDistance","weaponsAcquired"

In [14]:
def dropOutlier (df):
    df = df.dropna(axis=0)
    def dropIdx(df, idx) :
        df.drop(index=idx, inplace=True)
        dropIdx.dpIdx_sum +=len(idx)
        return df
    dropIdx.dpIdx_sum = 0

    print("Pre-Processing...")
    for i in df.columns.to_list() :
        df.drop(index=df[df[i].isnull()==True].index, inplace=True)
        
    print("Droping Outliers...")

    vip_features = ["assists","boosts","DBNOs","heals","kills","killStreaks","walkDistance", "revives", "roadKills", "vehicleDestroys"]

    group = df.groupby('groupId').count()
    df = dropIdx(df, df[df.groupId.isin(group[group["Id"]>group["Id"].quantile(0.9999)].index)==True].index) #수치고려 가능

    for col in (vip_features + ["damageDealt","longestKill", "rideDistance", "swimDistance","weaponsAcquired", "matchDuration"]):
        df = dropIdx(df, df[df[col]>df[col].quantile(0.999)].index)
    
    for col in vip_features:
        df = dropIdx(df, df[df["walkDistance"]<df[col]].index)

    df = dropIdx(df, df[df.groupby('matchId')['kills'].transform('max')  > df.groupby('matchId')['Id'].transform('count')  ].index)
    df = dropIdx(df, df[(df['rideDistance']==0) & (df['roadKills']>0)  ].index)

    #edge case
    df.loc[(df.maxPlace>1)&(df.numGroups==1), "maxPlace"] = 1

    print(f"{dropIdx.dpIdx_sum} Columns has deleted!") 

    del vip_features, group      
    gc.collect()
    
    return df

In [15]:
def feature_enginnering(df):
    # create columns
    print("Making columns...")
    stat_feature = ["assists","boosts","DBNOs","heals","kills","killStreaks","walkDistance", "revives", "roadKills", "vehicleDestroys","damageDealt","longestKill", "rideDistance", "swimDistance","weaponsAcquired"]
    stat_list = ["max","mean","median","min"]
    for col in stat_feature :
        for stat in stat_list:
            df[f"{col}_{stat}"] = df.groupby("groupId")[col].transform(stat)
            df[f"{col}_{stat}_Place"] = df.groupby("matchId")[f"{col}_{stat}"].transform('rank', ascending=False)
    print(len(stat_feature)*len(stat_list)+1, f"columns Made! now {len(df.columns)}column in DF.")
    
    # create kiilRank using matchId, kills 
    df["killRank"] = df.groupby("matchId")["kills"].rank("dense", ascending=False)
    
    # drop unnecessary columns 
    print('Dropping columns...')
    df = df.drop(["Id", "groupId","matchId"], axis=1)
    df = ordinal_encoding(df)
    df = df.drop(['killPlace'], axis=1)
    df = df.drop(['damageDealt'], axis=1)
    df = df.drop(['numGroups'], axis=1)
    df = df.drop(['killPoints','rankPoints','winPoints'], axis=1)
    
    print(len(df.columns), "columns in df")    
    reduce_mem_usage(df)
    del stat_feature
    gc.collect()
    return df

In [16]:
def sampling(df,n):
    idx = sorted(np.random.permutation(len(df))[:n])
    return df.iloc[idx].copy()
    n = len(train)  // 10
    df_sample = sampling(train, n)

## 전처리

In [17]:
train = ordinal_encoding(train)
test = ordinal_encoding(test)

In [17]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [18]:
train = train.dropna(axis=0)
test = test.dropna(axis=0)

In [20]:
train.isna().sum()

Id                 0
groupId            0
matchId            0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killPoints         0
kills              0
killStreaks        0
longestKill        0
matchDuration      0
matchType          0
maxPlace           0
numGroups          0
rankPoints         0
revives            0
rideDistance       0
roadKills          0
swimDistance       0
teamKills          0
vehicleDestroys    0
walkDistance       0
weaponsAcquired    0
winPoints          0
winPlacePerc       0
dtype: int64

## Simple Test

In [19]:
train = feature_enginnering(train)

Making columns...
61 columns Made! now 149column in DF.
Dropping columns...
141 columns in df


In [20]:
len(train.columns)

141

In [26]:
target = train['winPlacePerc']
features = train.drop(['winPlacePerc'], axis=1)

no parameter tunings learn: 0.0767996

- Linear 0.0661
- XGB 0.0584
- LGBM  0.0599
- CB  999:	learn: 0.0767948

- cb iterations=2, depth=2, learning_rate=1, loss_fuction='MAE' 1:	learn: 0.1097024
- cb iterations=2, learning_rate=1, loss_fuction='MAE' 1:	learn: 0.0842670
- cb iterations=2, learning_rate=10, loss_funtion="MAE" 1:	learn: 21.6615617
ㄴ learning rate is greater than 1. You probably need to decrease learning rate.
- iterations=2, depth=10, learning_rate=1, loss_function='MAE' 1:	learn: 0.0760201
- iterations=2, depth=20, learning_rate=1, loss_function='MAE' 1:	CatBoostError : Maximum tree depth is 16
- iterations=2, depth=16, learning_rate=1, loss_function='MAE' learn: 0.0701635

In [94]:
# AllModels_mae(target, features)

In [28]:
Linear_training_mae(target, features)

Linear Regression	 train=0.0661, val=0.0661


In [29]:
Lasso_training_mae(target, features)

Lasso		 train=0.1045, val=   0


In [35]:
test = feature_enginnering(test)

Making columns...
61 columns Made! now 149column in DF.
Dropping columns...
140 columns in df


In [31]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4446965 entries, 0 to 4446965
Columns: 141 entries, assists to killRank
dtypes: float16(106), int16(4), int8(31)
memory usage: 1.1 GB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1934174 entries, 0 to 1934173
Columns: 149 entries, Id to killRank
dtypes: float16(25), float64(81), int16(7), int8(32), object(4)
memory usage: 1.4+ GB


In [32]:
train_X, val_X, train_y, val_y = train_test_split(features, target, test_size=0.2, random_state=1511)
model = Lasso().fit(train_X,train_y)
pred_train2 = model.predict(train_X)
pred_val2 = model.predict(val_X)
mae_train2 = mean_absolute_error(train_y, pred_train2)
mae_val2 = mean_absolute_error(val_y, pred_val2)
print("Lasso\t\t train=%.4f, val=%4.f" % (mae_train2, mae_val2))


Lasso		 train=0.1045, val=   0


In [36]:
# train_X, test_X, train_y, test_y = train_test_split(features, target, test_size=0.2, random_state=1511)
result = model.predict(test)

from sklearn.

print("---------- Lasso ---------")
print('MSE in training: %.4f' % mean_squared_error(y_test, result))

1. 함수 수정
2. catboost (hyper parameter tuning)
3. best model test -> sumission -> 제출
4. ...